# Modelamiento

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.MLDS6_Assets.preprocessing import countFilesinFolder
from src.MLDS6_Assets.visualization import plotDataPartition
from src.MLDS6_Assets.models import trainTestSplitImages

## Partición de datos

Para la partición de datos, vamos a tomar de base los datos unidos y balanceados en un único dataset. Para realizar la partición, usaremos la función train test split de sklearn y mantener el esquema de carpetas

## Selección y diseño de modelos

Para abordar el problema de clasificación de imágenes en el diagnóstico de neumonía a partir de radiografías, hemos seleccionado los modelos VGG16 y EfficientNetB0, ambos disponibles en TensorFlow. Esta elección se justifica debido a que ambos modelos son redes neuronales convolucionales bien probadas en tareas de reconocimiento de imágenes y permiten capturar características visuales esenciales en radiografías. VGG16, con su arquitectura profunda y secuencial, es eficaz en detectar patrones sutiles y texturas asociadas a los signos de neumonía, como opacidades pulmonares; mientras que EfficientNetB0 se distingue por su capacidad de procesamiento optimizada y eficiente, lo que es ideal para implementaciones en tiempo real sin comprometer precisión. Además, se implementará transfer learning utilizando modelos preentrenados, lo que permitirá aprovechar características previamente aprendidas en grandes conjuntos de datos, acelerando el entrenamiento y mejorando la generalización en nuestro conjunto de datos específico. Al combinar ambos modelos, podemos lograr un balance entre precisión y velocidad, mejorando la fiabilidad diagnóstica. Además, ambos modelos se pueden adaptar añadiendo capas densas y técnicas como dropout o normalización para ajustarse mejor a las necesidades específicas del conjunto de datos, maximizando así el rendimiento en la tarea de clasificación.

### Carga de modelos pre entrenados y configuracion de mlops

A continuación, comenzamos definiendo las funciones necesarias para pre-cargar los modelos VGG16 y EfficientNetB0, los cuales serán modificados y adaptados para abordar el problema en cuestión. Es importante señalar que, aunque se crearán las funciones correspondientes, no se ejecutarán hasta el final. Esto se hará con el propósito de mantener un orden claro y ejecutar todo de manera estructurada en la última fase del proceso.

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow

# Configuración inicial de MLflow
mlflow.set_tracking_uri("mlruns/")
mlflow.tensorflow.autolog()  # Activar autologging para TensorFlow

# Función para cargar un modelo base preentrenado
def load_pretrained_model(model_name, input_shape):
    if model_name == 'VGG16':
        base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'EfficientNetB0':
        base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Model name not recognized. Choose 'VGG16' or 'EfficientNetB0'.")

    for layer in base_model.layers:
        layer.trainable = False

    return base_model

### Personalizacion de los modelos
Recordemos que nuestro objetivo es adaptar estos modelos a una arquitectura diferente para tener una mejor aproximacion al problema, por lo cual, generamos una funcion que genere dos configuraciones de capas densas al final de este que nos permita generar distintos modelos para comparar. Con esto, la idea es generar cuatro modelos entrenables diferentes que se puedan entrenar y evaluar

In [ ]:
# Definición de modelos personalizados
def config_model(base_model, version, lr=1e-3):
    input_layer = layers.Input(shape=(765, 500, 3))
    x = base_model(input_layer)
    x = layers.GlobalAveragePooling2D()(x)

    if version == 1:
        x = layers.Dense(512, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
    elif version == 2:
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(32, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
    else:
        raise ValueError("Version not recognized. Choose 1 or 2.")

    output_layer = layers.Dense(3, activation='softmax')(x)
    model = models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy'),
                 tf.keras.metrics.AUC(multi_label=True)]
    )
    return model

### Creacion de callbacks
se configuran funciones automáticas que se ejecutan durante el entrenamiento para optimizar el proceso. Estas incluyen guardar el mejor modelo (ModelCheckpoint), detener el entrenamiento si no hay mejora en un tiempo (EarlyStopping), y reducir el learning rate si el modelo deja de mejorar (ReduceLROnPlateau). Estas herramientas ayudan a evitar el sobreajuste y a ajustar el modelo de forma más eficiente.








In [ ]:
# Función para configurar callbacks
def create_callbacks(model_name):
    os.makedirs("checkpoints", exist_ok=True)
    checkpoint_path = f"checkpoints/best_{model_name}.h5"

    checkpoint = ModelCheckpoint(
        checkpoint_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1
    )

    early_stopping = EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1
    )

    return [checkpoint, early_stopping, reduce_lr]

### Entrenamiento del modelo usando Mlflow
Una vez definido todo lo anterior, definimos una funcion para que mlflow para que empiece a realizar los entrenamientos correspondientes

In [ ]:
# Función para entrenar el modelo
def train_model(model, model_name, train_data, validation_data, epochs=10):
    callbacks = create_callbacks(model_name)

    with mlflow.start_run(run_name=model_name):
        history = model.fit(
            train_data,
            epochs=epochs,
            validation_data=validation_data,
            callbacks=callbacks,
            verbose=1
        )

    return history

### Historial de entrenamiento
Es importante tambien destacar los historiales de entrenamiento, por lo cual se realiza el graficado tal como se ve a continuacion:

In [ ]:
# Función para visualizar el historial de entrenamiento
def plot_training_history(history, model_name):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history['accuracy'], label='train')
    ax1.plot(history.history['val_accuracy'], label='validation')
    ax1.set_title(f'{model_name} - Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    ax2.plot(history.history['loss'], label='train')
    ax2.plot(history.history['val_loss'], label='validation')
    ax2.set_title(f'{model_name} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.tight_layout()
    plt.savefig(f"{model_name}_training_history.png")
    plt.show()

### Ejecucion de todas las funciones anteriores

In [ ]:
input_shape = (765, 500, 3)

# Cargar modelos base
model_vgg16 = load_pretrained_model('VGG16', input_shape)
model_efficientnet = load_pretrained_model('EfficientNetB0', input_shape)

# Configurar modelos completos
model_vgg16_v1 = config_model(model_vgg16, version=1)
model_vgg16_v2 = config_model(model_vgg16, version=2)
model_efficientnet_v1 = config_model(model_efficientnet, version=1)
model_efficientnet_v2 = config_model(model_efficientnet, version=2)

# ACA
# ACA VAN CARGADOS LOS DATOS
# ACA
train_data = validation_data = None 

# Entrenamiento
history_vgg16_v1 = train_model(model_vgg16_v1, "VGG16_v1", train_data, validation_data)
history_vgg16_v2 = train_model(model_vgg16_v2, "VGG16_v2", train_data, validation_data)
history_efficientnet_v1 = train_model(model_efficientnet_v1, "EfficientNetB0_v1", train_data, validation_data)
history_efficientnet_v2 = train_model(model_efficientnet_v2, "EfficientNetB0_v2", train_data, validation_data)

# Visualización
plot_training_history(history_vgg16_v1, "VGG16_v1")
plot_training_history(history_vgg16_v2, "VGG16_v2")
plot_training_history(history_efficientnet_v1, "EfficientNetB0_v1")
plot_training_history(history_efficientnet_v2, "EfficientNetB0_v2")